In [ ]:
import google.generativeai as genai
from google.cloud import translate_v2 as translate
from google.cloud import texttospeech
import speech_recognition as sr

import pygame

In [ ]:
import torch
import sounddevice as sd

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")

In [ ]:
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
english='en'

In [ ]:
model = genai.GenerativeModel('gemini-pro')

In [ ]:
def record_audio(file):
    r = sr.Recognizer()
    with sr.AudioFile(file) as source:
        audio = r.record(source)
    try:
        print("Sphinx thinks you said " + r.recognize_sphinx(audio))
    except sr.UnknownValueError:
        print("Sphinx could not understand audio")
    except sr.RequestError as e:
        print("Sphinx error; {0}".format(e))
    return r.recognize_sphinx(audio)

In [ ]:
def play_audio(file):
    sound=pygame.mixer.Sound(file)
    recordlength=int(sound.get_length()*1000)
    sound.play()
    pygame.time.wait(recordlength)

In [ ]:
def make_speech_file(speech_file_path,text):
    googclient = texttospeech.TextToSpeechClient()
    synthesis_input = texttospeech.SynthesisInput(text=text)
    voice=texttospeech.VoiceSelectionParams(
        language_code=lang, ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL
    )
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3
    )
    response = googclient.synthesize_speech(
        input=synthesis_input, voice=voice, audio_config=audio_config
    )
    with open(speech_file_path, "wb") as out:
        out.write(response.audio_content)
        print('Audio content written to file "output.mp3"')

In [ ]:
def translate_to(lang,text):
    transclient=translate.Client()
    if isinstance(text,bytes):
        text=text.decode('utf-8')
    result=transclient.translate(text,target_language=lang)
    return result['translatedText']

In [ ]:
def generate_response(input_text):
    prompt='Please respond logically to the following sentence in a conversation: '+input_text
    generation=model.generate_content(prompt).text
    #print(generation)
    return generation

In [ ]:
def machine_turn(text):
    machine_file='recordings/machine.wav'
    if text=='':
        eng='Hello, how are you today?'
        talk=translate_to(lang,eng)
    else:
        talk=generate_response(text)
    make_speech_file(machine_file,talk)
    eng=translate_to(english,machine_file)
    play_audio(machine_file)
    print('Computer: '+talk+' ('+eng+')')

In [ ]:
def human_turn():
    file='recordings/human.wav'
    talk=record_audio(file)
    eng=translate_to(english,file)
    print('Me: '+talk+' ('+eng+')')
    return talk

In [ ]:
def conversation():
    human_response=''
    x=0
    while x!=5:
        machine_turn(human_response)
        human_response=human_turn()
        x+=1

In [ ]:
lang='en'

In [ ]:
pygame.init()

In [47]:
conversation()

Audio content written to file "output.mp3"
Computer: Hello, how are you today? (recordings/machine.wav)


ValueError: Audio file could not be read as PCM WAV, AIFF/AIFF-C, or Native FLAC; check if file is corrupted or in another format

In [ ]:
pygame.quit()